## Generate SQL query

In [3]:
#r "nuget: Microsoft.SemanticKernel, 1.5.0"
#r "nuget: Microsoft.SemanticKernel.Plugins.Memory, 1.5.0-alpha"
#r "nuget: System.Linq.Async, 6.0.1"

#!import ../config/Settings.cs

using Microsoft.SemanticKernel;
using Kernel = Microsoft.SemanticKernel.Kernel;

var (useAzureOpenAI, model, azureEndpoint, apiKey, bingApiKey, orgId) = Settings.LoadFromFile();
var _kernel = Microsoft.SemanticKernel.Kernel.CreateBuilder()
            .AddAzureOpenAIChatCompletion(
                model,   // deployment name
                azureEndpoint, // Azure OpenAI Endpoint
                apiKey)      // Azure OpenAI Key
            .Build();

Installed Packages Microsoft.SemanticKernel, 1.5.0 Microsoft.SemanticKernel.Plugins.Memory, 1.5.0-alpha System.Linq.Async, 6.0.1

Build memory

In [6]:
using Microsoft.SemanticKernel.Memory;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Plugins.Memory;

#pragma warning disable SKEXP0011, SKEXP0003, SKEXP0052
var memoryBuilder = new MemoryBuilder();
memoryBuilder.WithAzureOpenAITextEmbeddingGeneration(
        "mrtextembeddingada002",
        azureEndpoint, 
        apiKey)
    .WithMemoryStore(new VolatileMemoryStore());   
var _memory = memoryBuilder.Build();
_kernel.ImportPluginFromObject(new TextMemoryPlugin(_memory));

Database schema

In [12]:
const string MemoryCollectionName = "tables";

await _memory.SaveInformationAsync(MemoryCollectionName, id: "orders", text: 
@"Table name: orders
Columns in order table: id, customerId, date, total");
await _memory.SaveInformationAsync(MemoryCollectionName, id: "customer", text: 
@"Table: customers
Columns in customers table: id, name, address");

Define system prompt

In [14]:
const string skPrompt = @"
Generate SQL query based on user input or ask for more details in case you need more information to generate the query. The generated query must specify names of columns to return rather than using the ""*"" (asterisk) operator.
Use the schema to ensure only columns listed for each table are used in the query. Do not use column names not in the schema.
If you don't have enough information for SQL query generation - respond with your question starting with ""ChatBot: "" prefix. For example: ""ChatBot: What details do you need about your customer?"".
If you have enough information for SQL query generation - generate a query and return it starting with ""SQL: "" prefix. For example: ""SQL: SELECT FirstName, LastName FROM Contacts"". 
If the user input does not give you enough information about which columns to use in the query, respond with your question starting with ""ChatBot:"". 

Chat: {{$history}}
User input: {{$userInput}}
Schema: {{recall $userInput}}

###
# The following examples are for the SQLCopilot plugin

userInput: Show customers in New York
chatbot: ChatBot: What specific data do you want about these customers?

userInput: List orders worth more than $100
chatbot: ChatBot: what data do you need about those orders?

userInput: List names of customers in Boston
chatbot: SQL: SELECT name FROM customers WHERE city = 'Boston'

userInput : List order ids for customer 123?
chatbot: SQL: SELECT id FROM orders WHERE customerId = 123

userInput: Who ordered product XYZ?
chatbot: ChatBot: what data do you need about that customer?

User: {{$userInput}}
ChatBot: ";

Questions

In [9]:
string[] queries = new string[]
{
    "Show customers in Boston",
    "List orders worth more than $100",
    "List names of customers in Boston",
    "List orders ids for customer 123",
    "Who ordered product XYZ?",
    "I need order data"
};

Conduct conversation

In [13]:
#pragma warning disable SKEXP0003
StringBuilder history = new StringBuilder();

var chatFunction = _kernel.CreateFunctionFromPrompt(skPrompt, new OpenAIPromptExecutionSettings { MaxTokens = 200, Temperature = 0.8 });
#pragma warning disable SKEXP0052
var arguments = new KernelArguments();
arguments[TextMemoryPlugin.CollectionParam] = MemoryCollectionName;
arguments[TextMemoryPlugin.LimitParam] = "2";
arguments[TextMemoryPlugin.RelevanceParam] = "0.9";
foreach(var userInput in queries)
{
    //Console.Write('>');
    //var userInput = Console.ReadLine();
    //if (String.IsNullOrEmpty(userInput))
    //{
    //    break;
    //}
    Console.WriteLine($">{userInput}");
    arguments["userInput"] = userInput;
    var answer = await chatFunction.InvokeAsync(_kernel, arguments);
    history.Clear();
    while(answer.ToString().StartsWith("ChatBot:"))
    {
        Console.WriteLine(answer);
        var inp = await InteractiveKernel.GetInputAsync(answer.ToString());
        Console.WriteLine($">> {inp}");
        var result = $"\nUser: {userInput}\nChatBot: {answer}\n";
        history.Append(result);
        arguments["history"] = history;
        arguments["userInput"] = inp;
        answer = await chatFunction.InvokeAsync(_kernel, arguments);
    }
    Console.WriteLine(answer);
};

>Show customers in Boston
ChatBot: What specific data do you want about these customers?
>> names
SQL: SELECT FirstName, LastName FROM Customers WHERE City = 'Boston'
>List orders worth more than $100
ChatBot: What specific data do you need about those orders?


Error: Command cancelled.